# Tarea 4: Redes Convolucionales <br/> CC6204 Deep Learning, Universidad de Chile 

**Fecha de entrega: 11 de diciembre de 2020 ([Hoja de respuestas](https://colab.research.google.com/drive/1mD5IyGRDQqg3Mxh6TqFQEGWJDY7TVUCF#scrollTo=iTY5bha_xgCj))**

En esta tarea programarás arquitecturas de redes convolucionales y las probarás para clasificación en lo datos de [CIFAR10 y CIFAR100](https://www.cs.toronto.edu/~kriz/cifar.html) además de una tarea de Subtitulado de Imágenes (*Image Captioning*) con datos de [Filckr8k](http://hockenmaier.cs.illinois.edu/8k-pictures.html) introducido en [este artículo](https://www.aclweb.org/anthology/W10-0721/).

El material necesario para resolver esta tarea es el siguiente:
* [Video: Introducción a las Redes Convolucionales](https://www.youtube.com/watch?v=vSHSmiKiiDw)
* [Video: Pooling, AlexNet y VGG](https://www.youtube.com/watch?v=ju7nKaFaFvc)
* [Video: InceptionNet (GoogLeNet)](https://www.youtube.com/watch?v=AxWG1aLWODE)
* [Video: ResNet y DenseNet](https://www.youtube.com/watch?v=C7S7wBsg2KE)
* [Notebook: Redes Convolucionales en `pytorch`](https://drive.google.com/file/d/1n7za4e0ihSYhKiTnHFNdynl752fW2rCB/view?usp=sharing)

Te recomendamos fuertemente que mires con cuidado el Notebook mencionado arriba antes de comenzar con la tarea (posiblemente sea bueno que lo mires y te familiarices con él antes de seguir leyendo).

Adicionalmente, para la parte donde debes elegir alguna de las arquitecturas,  puede ser de ayuda consultar los siguiente artículos:
* [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556) (VGG, ICLR 2015) 
* [Going Deeper With Convolutions](https://arxiv.org/abs/1409.4842v1) (GoogLeNet, CVPR 2015)
* [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) (ResNet, CVPR 2016)
* [Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993) (DenseNet, CVPR 2017)

(por Jorge Pérez, https://github.com/jorgeperezrojas, y Jesús Pérez, https://github.com/jssprz)

# Preliminares: Redes Convolucionales en `pytorch`

En esta tarea empezaremos a usar redes de arquitecturas más complejas por lo que dejaremos todo el trabajo de cálculo de backpropagation y optimización a `pytorch`. Además usaremos las capas de convolución y pooling ya implementadas en `pytorch`. Para esta tarea puedes usar cualquiera de las clases de:

* [`torch.nn`](https://pytorch.org/docs/stable/nn.html) para funcionalidades para capas convolucionales, pooling, dropout, batch normalization, funciones de activación, pérdida, etc.
* [`torch.optim`](https://pytorch.org/docs/stable/optim.html) para funcionalidades de optimizadores como SGD, Adam, RMSProp,
* [`torch.utils.data`](https://pytorch.org/docs/stable/data.html) para funcionalidades para cargar datos, y
* [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html) con funcionalidades para cargar datos, en particular CIFAR10 y CIFAR100.

Para guiarte en cómo usar estas funcionalidades y un ejemplo simple para CIFAR10, puedes ver el notebook de [Redes Convolucionales en `pytorch`](https://colab.research.google.com/drive/1n7za4e0ihSYhKiTnHFNdynl752fW2rCB). 
Te recomendamos estudiar también la documentación para aprender detalles.

# Instrucciones Generales

En esta parte de la tarea debes progamar dos arquitecturas de redes convolucionales. Una de estas arquitecturas será [GoogLeNet](https://arxiv.org/abs/1409.4842v1). Para la otra, te damos la posibilidad de escoger entre las mencionadas en el inicio de la tarea (VGG, ResNet, DenseNet). En la [Hoja de Respuestas](https://colab.research.google.com/drive/1mD5IyGRDQqg3Mxh6TqFQEGWJDY7TVUCF#scrollTo=iTY5bha_xgCj) debes programar ambas arquitecturas completas usando las funcionalidades de `pytorch`.

Posteriormente, usaremos ambas arquitecturas para resolver dos problemas independientes:
1. Clasificación de Imágenes en CIFAR10 (Parte 1)
2. Subtitulado de Imágenes (Image Captioning) mediante Recuperación de Textos en Flickr8k (Parte 2)

Para ambos problemas la entrada serán imágenes de 32x32 con tres canales de colores, por lo que tu red debe recibir (en el método `forward`) un tensor de dimensiones `(B,3,32,32)` donde `B` es el tamaño del batch. Nota que esto cambiará un poco las arquitecturas estándar de las redes (pues ellas reciben como entrada imágenes de 224x224). Más abajo te damos detalle de cómo aboradar esto.

Para el caso de las salidas es un poco diferente dependiendo del problema (clasificación o subtitulado). 
Para asegurar que las redes que construyas estén preparadas para ambos problemas, la salida (o sea, lo que entrega la función `forward`) deben ser (al menos) dos tensores que llamaremos `logits` y `hidden`, y los debes entregar como parte de un diccionario. La idea de cada uno la explicamos a continuación: 
- El tensor `logits` es la salida típica de una red de clasificación **antes de pasar** por la función `softmax` y debe tener dimensiones `(B, n_classes)` donde `n_classes` es la cantidad de clases para la que se usará la arquitectura. El valor `n_classes` debe ser indicado en el inicializador de la clase donde programes tu red. En este caso entregamos *logits* en vez de nuestro típico $\hat{y}$ por la forma en que pytorch implementa el [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Para el caso de GoogLeNet debes considerar dos salidas adicionales que explicaremos después.
- Por su parte el tensor `hidden` debe tener dimensiones `(B, h_size)` y corresponde a **alguna capa escondida de tu red** que debes elegir para el problema de subtitulado de imágenes. El cómo usaremos este tensor te quedará claro más adelante en el enunciado. 

**Para el problema 1.** Te recomendamos revisar los artículos de cada arquitectura para ver cuál es la mejor configuración de cada una. Intenta alcanzar al menos un 80% de acierto en el conjunto de prueba de CIFAR10. El notebook de [Redes Convolucionales en `pytorch`](https://drive.google.com/file/d/1n7za4e0ihSYhKiTnHFNdynl752fW2rCB/view?usp=sharing) te guiará en cómo usar CIFAR10 directamente en `pytorch`. Para el entrenamiento de los modelos puedes usar la función auxiliar `train_for_clasification` implementada en el [repositorio del curso](https://github.com/dccuchile/CC6204/blob/master/2020/tareas/tarea4/utils.py) (y que incluimos en la hoja de respuesta).

**Para el problema 2.** No te pediremos que encuentres la mejor arquitectura para resolver el problema ni el mejor resultado. La idea más bien es que reutilices tus arquitecturas en un problema bastante más complejo que el anterior y con una nueva función de pérdida que describiremos más adelante. Nota que para este problema el valor `n_classes` no cumplirá ninguna función. Para el entrenamiento de los modelos puedes usar la función auxiliar `train_for_retrieval` implementada en el [repositorio del curso](https://github.com/dccuchile/CC6204/blob/master/2020/tareas/tarea4/utils.py).

Aunque las funciones `train_for_clasification` y `train_for_retrieval` te dan alguna flexibilidad como la de incluir un learning-rate scheduler en el entrenamiento, es posible que te lleguen a resultar demasiado sencillas en algún punto. Si deseas usar técnicas más avanzadas y/o nuevas métricas, siéntente libre de incluir una nueva implementación de estas funciones en tu cuaderno y/o hacer un `pull-request` en el repositorio del curso para incluir tus mejoras.

# Parte 1: Arquitectura Convolucional GoogLeNet (y otras) para CIFAR10

La arquitectura GoogLeNet se basa en una arquitectura de red neuronal convolucional profunda denominada "InceptionNet", que fue responsable de establecer el nuevo estado del arte para la tarea de clasificación y detección en la competencia *ImageNet Large-Scale Visual Recognition Challenge 2014 (ILSVRC 2014)*. En esta parte harás tu propia implementación de GoogLeNet (1b) y además implementarás otra arquitectura a tu elección (1c). 

## 1a) Inception Module

Empezaremos implementando el *Inception Module* como una pequeña red de pytorch (heredando de `nn.Module`). La construcción de estos módulos puedes revisarla en [este video (minuto 23:40)](https://youtu.be/AxWG1aLWODE?t=1428). En resumen un Inception Module está compuesto por una primera fase en donde dos capas convolucionales de 1x1 son aplicadas a la entrada. Llamaremos $C_1$ y $C_2$ a estas capas convolucionales. La salida de $C_1$ se entrega como entrada a una capa convolucional de 3x3 y la salida de $C_2$ a una capa convolucional de 5x5. Llamemos $D_1$ y $D_2$ a estas capas, respectivamente. Además el Inception Module tiene una capa de Max Pooling de 3x3 que se aplica a la entrada seguida de una capa convolucional de 1x1 que llamaremos $D_3$, mas una capa convolucional adicional de 1x1 que llamaremos $D_4$ y que se aplica directamente a la entrada. Las salidas de $D_1$, $D_2$, $D_3$ y $D_4$ se concatenan (en ese orden) para generar la salida final del Inception Module.

Un punto importante es que todas las convoluciones y el pooling tienen stride $1$ y el padding necesario para mantener las dimensiones de alto y ancho de la entrada.

Como se menciona en el video, un Inception Module queda totalmente definido por la cantidad de canales de entrada más la cantidad de canales de salida de cada una de las capas convolucionales. Para simplificar la notación usaremos los siguientes nombres para la cantidad de canales de salida de cada una de las capas: 
- `ch_3x3_reduce`: cantidad de canales de salida de la capa $C_1$, 
- `ch_5x5_reduce`: cantidad de canales de salida de $C_2$, 
- `ch_3x3`: cantidad de canales de salida de $D_1$, 
- `ch_5x5`: cantidad de canales de salida de $D_2$, 
- `ch_pool_proj`: cantidad de canales de salida de $D_3$ y
- `ch_1x1`: cantidad de canales de salida de $D_4$.  

Estos nombres son similares a los que se usan en la Tabla 1 del [artículo original de GoogleNet](https://arxiv.org/abs/1409.4842v1). Considerando lo anterior, debes implementar el Inception Module con una clase como la que sigue.

In [ ]:
class InceptionModule(nn.Module):
  def __init__(self, 
               in_channels, 
               ch_3x3_reduce=96, 
               ch_5x5_reduce=16,
               ch_3x3=128,
               ch_5x5=32,
               ch_pool_proj=32,
               ch_1x1=64
    ):
    super(InceptionModule, self).__init__()
    # Acá inicializa todos los parámetros
    ...

  def forward(self, x):
    # Calcula la salida como un tensor con cantidad de canales de
    # salida dado por ch_3x3 + ch_5x5 + ch_pool_proj + ch_1x1
    ...

    return ....

## 1b) GoogLeNet

En esta parte debes usar tu implementación anterior para implementar una versión completa de GoogLeNet. Hay un par de detalles que es bueno que consideres. Primero recuerda que una de las características de GoogLeNet es que usa una función de pérdida compuesta por un promedio de tres pérdidas obtenidas desde capas intermedias de la red, además de la capa final usual. Para esto, debes considerar una variable `use_aux_logits` en el inicializador que especifica si se debe o no retornar los vectores para clasificación intermedia. La función `forward` es la que se debe encargar de entregar estos vectores que debes retornar en el campo `aux_logits` del diccionario de salida tal como indica el esqueleto de implementación de abajo.

Adicionalmente debes considerar que las imágenes que recibirá tu red serán de dimensiones `(3,32,32)` por lo que debes adaptar la arquitecutra para no trivializar los tensores con las primeras capas. Te proponemos considerar las primeras convoluciones de la arquitectura estándar de GoogLeNet con *stride* 1, de manera de no bajarle la resolución antes de ingresar al primer Inception Module. Pero en general eres libre de decidir cómo modificar el input al inicio. Simplemente asegúrate que al primer Inception Module entra un tensor con dimensiones similares a 28 x 28 que son los que considera la arquitectura original.



In [ ]:
class GoogLeNet(nn.Module):
  def __init__(self, n_classes, use_aux_logits=True):
    super(GoogLeNet, self).__init__()

    # Define las capas de convolución y pooling de GoogLeNet
    ...

    # Decide si usar la clasificación auxiliar
    self.use_aux_logits = use_aux_logits
    if self.use_aux_logits:
      # Acá lo que necesites inicializar en este caso
      ...

    # Capa de salida (antes de la función de salida)
    self.fc_out = nn.Linear(..., n_classes)

  def forward(self, x):
    # Crea una lista para los logits auxiliares si fuera necesario
    if self.use_aux_logits and self.training:
      aux_logits = []
    else:
      aux_logits = None

    # Computa las representaciones internas de la red
    ...

    # Si se usa la clasificación auxiliar, computa logits auxiliares
    if self.use_aux_logits and self.training:
      aux_logit_1 = ...
      # Agrégalo a la lista de logits auxiliares
      aux_logits.append(aux_logit_1)

    # Continúa computando las representaciones internas de la red
    ...

    # Si se usa la clasificación auxiliar, computa logits auxiliares
    aux_logit_2 = None
    if self.use_aux_logits and self.training:
      aux_logit_2 = ...
      # Agrégalo a la lista de logits auxiliares
      aux_logits.append(aux_logit_2)

    # Continúa computando las representaciones internas de la red
    ...

    # N x out_size
    logits = self.fc_out(...)

    # En hidden debes devolver alguna de las capas oculta de la red
    return {'hidden': ..., 'logits': logits, 'aux_logits': aux_logits}

## 1c) Arquitectura convolucional a elección

En esta parte debes decidir qué arquitectura adicional quieres programar. Trata de hacer una implementación modular implementando pequeños trozos reusables de las arquitecturas. Por ejemplo, si te animas a implementar la red DenseNet, deberías hacer un modulo para el *Dense Block* que puedas reusar varias veces dentro de la arquitectura. Trata de encontrar la mejor configuración posible. Los artículos base de cada arquitectura te pueden ayudar a encontrarla. Por ejemplo, en la Tabla 2 del [artículo original de DenseNet](https://arxiv.org/abs/1608.06993), mencionan una configuración de la arquitectura que alcanza un 5.19% de error en CIFAR10.  

In [ ]:
# Acá el código para tu primera arquitectura

class ...(nn.Module):
  def __init__(self, n_classes):
    super(..., self).__init__()

    # Define las capas de convolución y pooling de tu arquitectura
    ...

    # Capa de salida (antes de la función de salida)
    self.fc_out = nn.Linear(..., n_classes)

  def forward(self, x):
    # Computa las representaciones internas de la red
    ...

    # N x out_size
    logits = self.fc_out(...)

    # En hidden debes devolver alguna de las capas oculta de la red
    return {'hidden': ..., 'logits': logits}

## 1d) Clasificación de Imágenes en CIFAR10

En esta parte debes probar tus dos arquitecturas. Incluye gráficos del entrenamiento para cada una y busca algunos hiperparámetros para obtener un buen acierto. Puedes basarte en el [Notebook de Redes Convolucionales en `pytorch`](https://drive.google.com/file/d/1n7za4e0ihSYhKiTnHFNdynl752fW2rCB/view?usp=sharing) para crear los datasets correspondientes. También dejamos código en la hoja de respuestas para llegar y probar.

Entrega al menos dos curvas de entrenamiento y muestra una grilla con un conjunto de imágenes y las predicciones que entrega tu mejor red.

In [ ]:
# En la Hoja de Respuestas encontrarás código para aydarte en esta parte.

## 1e) Opcional: CIFAR100

Usa alguna de tus arquitecturas (o si quieres, ambas) para que procesen el conjunto de datos CIFAR100 usando la [clase de `pytorch` para este dataset](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.CIFAR100) y reporta tus resultados.

In [ ]:
# Acá tu código

# Parte 2: Subtitulado de Imágenes mediante Recuperación de Textos

En esta parte de la tarea usaremos las redes convolucionales para recuperar las descripciones textuales (en inglés) más similares a una imagen cualquiera, considerando un conjunto de descripciones posibles. Para esto, entrenaremos un modelo capaz de proyectar las imágenes y las descripciones en un espacio $d$-dimensional, de modo que se cumplan ciertas condiciones. Como el procesamiento de textos no es parte de esta tarea, les brindaremos codificaciones de los textos (vectores de características) precomputadas.

Como ya debes saber, al usar como última capa de $d$ neuronas completamente conectada, el modelo garantiza que la salida sea un vector de codificación (encoding) de tamaño $d$. Usaremos las codificaciones para comparar las imágenes con los textos mediante una función de distancia, de la siguiente manera:
[visual-text-emedding (imagen)](https://drive.google.com/file/d/1P9A028DXRi3P5NKM5KSCisicanTQRnDh/view?usp=sharing)

Formalmente, para una imagen $x$ y un texto $w$, denotamos las funciones de encoding $f(x)$ y $g(w)$ que producen vectores en un mismo espacio $\mathbb{R}^d$. La función $f$ se computa mediante un modelo basado en CNN seguido de una red feed forward fully connected (MLP) que tiene como salida un vector en $\mathbb{R}^d$. Mientras que la fúncion $g$ recibirá como entrada vectores precomputados que representan textos y mediante otro MLP los proyecta en $\mathcal{R}^d$. Las dos redes MLP solo se usan para proyectar los vectores y pueden ser tan simples como una única capa sin siquera un función de activación. 

## 2a) Codificación de Imágenes y Textos

Comenzaremos implementando las codificaciones mencionadas arriba: la codificación de imágenes en el módulo `ImageEncoding` y de texto en el módulo ` TextEncoding`. Para la codificación de imágenes lo que haremos será usar una red convolucional de las que ya implementaste y que debe ser entregada en el inicializador (variable `cnn_model`) junto con la dimensión de la salida intermedia de esa red (el valor `h_size` que mencionamos al inicio de la tarea) en la variable `cnn_out_size`. Para el caso de las imágenes de entrada estas seguirán teniendo dimensiones `(3,32,32)` por lo que el input de la función `forward` debe ser `(B,3,32,32)`. Para el caso del texto debes suponer que estos vendrán codificados como un vector de dimensión `text_embedding_size`. Ese valor debe ser indicado en el inicializador de la clase ` TextEncoding` (en general en esta tarea usaremos `4096` dimensiones). La función `forward` entonces debe estar preparada para recibir un tensor de dimensiones `(B, text_embedding_size`).

In [ ]:
class ImageEncoding(nn.Module):
  def __init__(self, cnn_model, cnn_out_size, out_size=128):
    super(ImageEncoding, self).__init__()
    self.cnn_model = cnn_model

    # Defina las capas de su MLP
    # Hints: no usar más de 3 capas
    #        incorpora alguna técnica de regularización que ya conoces
    ...

  def forward(self, x):
    x = self.cnn_model(x)['hidden']

    # Compute las capas de su MLP
    ...

    # En fc_out debe almacenar el encoding en R^d
    return {'logits': ...}

In [ ]:
class TextEncoding(nn.Module):
  def __init__(self, text_embedding_size=4096, out_size=128):
    super(TextEncoding, self).__init__()

    # Defina las capas de su MLP
    # Hints: no usar más de 3 capas
    #        incorpora alguna técnica de regularización que ya conoces
    ...

    self.use_last_bn = use_last_bn
    if use_last_bn:
      self.bn = nn.BatchNorm1d(out_size)

  def forward(self, x):
    # Compute las capas de su MLP
    ...

    # En logits debe almacenar el encoding en R^d
    return {'logits': ...}

## 2b) Buenas codificaciones y la *Triplet Loss*

Lo que necesitamos en esta parte de la tarea es que nuestra red aprenda una "buena codificación" tanto para imágenes como para textos. En nuestro caso una codificación es buena si:
- Las codificaciones de una imagen y los texto relacionados a su contenido son bastante similares entre sí,
- Las codificaciones de una imagen y los textos asociados a otras imágenes son muy diferentes.

Nos aseguraremos de que nuestras representaciones sean buenas mediante una función de pérdida que formaliza esta intuición. La función que usaremos se llama *triplet loss* e intenta "atraer" las codificaciones de una imagen y su texto para ubicarlas más cerca en $\mathbb{R}^d$, mientras que "repele" las codificaciones de una imagen y los otros textos del batch, para ubicarlas más separadas. 

## Triplet loss 

Su nombre viene dado a que se basa en tripletas $(A, P, N)$ donde:
- $A$ es una imagen "ancla",
- $P$ es un texto "positivo", es decir, un texto que se corresponde con el contenido visual de la imagen ancla, y
- $N$ es un texto "negativo", es decir, un texto que no se corresponde con el contenido visual de la imagen ancla.

Estas tripletas las obtendremos a partir de los datos de cada batch. En particular, cada batch tendrá un conjunto $B$ de pares $(A_1,P_1), (A_2,P_2), \ldots, (A_B,P_B)$. El texto negativo $N_i$ para la imagen ancla $A_i$ será algúno de los valores $P_j$ con $j\neq i$. La elección de este $N_i$ es bien importante y daremos detalles después, pero por ahora podemos suponer que efectivamente tenemos triples que se obtienen desde un batch de datos.

Como queremos asegurarnos de que $A_i$ esté más cerca del texto $P_i$ que de cualquier texto en el batch, usaremos un "margen" que llamaremos $\alpha$ de manera tal que:

$$\| f(A_i) - g(P_i) \|_2 + \alpha\;\; \lt \;\; \| f(A_i) - g(N_i) \|_2$$

donde $\| \cdot \|_2$ es la norma Euclideana de un vector. 
Para convertir esta restricción en un problema de optimización, podemos crear la siguiente función de costo (llamada "triplet cost") para cada triplete del batch:

$$\mathcal{J}_i = \max\bigg\{\, 0,\; \underbrace{\mid \mid f(A_i) - g(P_i) \mid \mid_2}_\text{(a)} - \underbrace{\mid \mid f(A_i) - g(N_i) \mid \mid_2}_\text{(b)} + \alpha \bigg\} \tag{1}$$

donde se cumple lo siguiente:
- El término (a) es la distancia Euclideana entre el ancla $A_i$ y el positivo $P_i$ para un triplete dado (queremos que esto sea pequeño).
- El término (b) es la distancia Euclideana entre el ancla $A_i$ y el negativo $N_i$ para un triplete dado (queremos que sea relativamente grande, por lo que tiene sentido tener un signo negativo antes de este término).
- El valor $\alpha$ actúa como *margen*. Es un parámetro que debemos elegir manualmente. Te recomendamos probar con $\alpha = 0.2$.

La función de loss será entonces el promedio de los costos $\mathcal{J}_i$ para todos los triples del batch de ejemplos. Con esta loss podemos usar backpropagation para ajustar todos los parámetros de la red y obtener entonces buenas codificaciones para imágenes y textos.

En esta parte debes implementar la funcion `TripletLoss` como un módulo de pytorch. La función debe recibir un conjunto de $B$ imágenes ancla (variable `anchor` en la función `forward` abajo), y un conjunto de $B$ representaciones de textos (`positive`) correspondientes a cada una de las imágenes y entregar el valor correspondiente a la Triplet Loss.

¿Cómo elegir el $N_i$? Hay varias formas de elegir el valor del texto negativo para cada imagen ancla. Una elección simple es elegir el texto que en el batch en curso tiene el mayor "triplet cost" con la representación de la imagen ancla. Otra forma alternativa sería simplemente considerar un ejemplo al azar del batch (distinto al texto de la imagen ancla). Finalmente podríamos también considerar *todos* los otros textos como negativos, en cuyo caso la cantidad de triples no sería $B$ si no $B(B-1)$ (para cada una de las $B$ imágenes ancla tendríamos $(B-1)$ textos negativos). Para decidir la forma en que elegiremos los textos negativos usaremos una variable `negative` en el inicializador de la clase que puede tomar los valores `max`, `random` o `all` y debes considerar en tu implementación. 

In [ ]:
# En la hoja de respuestas te damos algunas ayudas de cómo sería bueno 
# implementar esta parte, pero tu eres libre de generar tu implementación
# como mejor te parezca

class TripletLoss(nn.Module):
  def __init__(self, margin=.2, negative='all'):
    pass

  def forward(self, anchor, positive):
    pass

## 2c) Probando tu implementación en Flickr8k

En esta parte usaremos toda la implementación anterior para entrenar la red que selecciona el mejor subtítulo para cada imágen usando el dataset [Flickr8k](). Para simplificar el procesamiento de los datos de esta parte (y de la siguiente), en el repositorio del curso en el archivo [utils.py](https://github.com/dccuchile/CC6204/blob/master/2020/tareas/tarea4/utils.py) hemos dejado la clase `ImageCaptionDataset` que genera un dataset de pares `(img, txt_enc)` donde `img` es un tensor de dimensiones `(3,32,32)` que representa a una imagen y `txt_enc` es un vector de `4096` componentes con una representación del texto de descripción asociado a la imagen. En la [Hoja de Respuesta](https://colab.research.google.com/drive/1mD5IyGRDQqg3Mxh6TqFQEGWJDY7TVUCF#scrollTo=iTY5bha_xgCj) encontrarás código para descargar el dataset de Flickr8k pre-procesar los datos y crear un `ImageCaptionDataset` correspondiente a los conjuntos de train, test y validación. Con esto podrás comenzar el entrenamiento y testeo de tu red para este problema. Recuerda que puedes usar el procedimiento `train_for_retrieval` que también hemos dejado para ti en el repositorio del curso. 

Muestra las curvas de aprendizaje para tu arquitectura que de mejores resultados. En la Hoja de Respuestas también encontrarás algunas funcionalidades para visualizar cómo se comporta tu solución para distintas imágenes del conjunto de test. 

In [ ]:
# En la Hoja de respuestas encontrarás código para descargar y pre-procesar
# el conjunto de datos y para armar un dataset que luego puedas usar para
# entrenar.

## 2d) Opcional: COCO Captions

En esta parte puedes probar tu implementación en un conjunto de datos bastante más desafiante: [COCO Captions](https://cocodataset.org/#captions-2015). En la Hoja de Respuesta dejamos código para que puedas probar de manera relativamente simple tus arquitecturas en este dataset. 


In [ ]:
# En la Hoja de respuestas encontrarás código para descargar y pre-procesar
# el conjunto de datos y para armar un dataset que luego puedas usar para
# entrenar.